## <center>Preprocesamiento de datos para **World University Rankings 2023</center>**


<div style="text-align: justify;">
Como primer paso, se importó el conjunto de datos usando la librería <b>Pandas</b> de Python. Posteriormente se hizo una pequeña inspección de los mismos para tener una idea de su magnitud.
</div>


In [2]:
import pandas as pd

data = pd.read_csv("/home/danielhz19/Documents/VSCode/UCV/ELECTIVAS/DM/Practica/Asignacion1/World University Rankings 2023.csv")
df = pd.DataFrame(data)


print(data.shape)

(2341, 13)


<div style="text-align: justify;">
Dado la gran cantidad de filas en el conjunto de datos, se decidió inspeccionar mas a fondo usando Excel como herramienta de visualización.
</div>

![Captura de pantalla al momento de importar los datos en Excel](/home/danielhz19/Documents/VSCode/UCV/ELECTIVAS/DM/Practica/Asignacion1/images/DM_Asig1_ExcelDataImport.PNG)

<div style="text-align: justify;">
Se detectaron varios problemas funcionales, como pueden ser la inconsistencia de los valores en la columna "University Rank", dado que a partir de la observación 200, dichos valores se convierten en un rango en lugar de un valor específico y la existencia del valor "Reporter", a pesar de ser valores validos dentro del sistema mundial de ranking universitario, pueden presentar problemas al momento de procesar los datos. De igual forma ocurre con el campo "Name of University" puesto que hay observaciones con el valor: <b>NaN</b>, cabe destacar que algunas de estas observaciones poseen datos en las demás columnas, sin embargo, el nombre de una universidad no es algo que podamos estimar con dichos datos. en la columna destinada a almacenar la ubicación de la universidad también existe un problema funcional, puesto que los campos vacíos podrían afectar al momento de procesar los datos.
</div>


<div style="text-align: justify;">
Como primera accion, se filtraron las observaciones que tenian en la columna "Name of University" un valor invalido, es decir, NaN o directamente un campo vacio.
</div>

In [4]:
invalid_names = df['Name of University'].isna()
df.drop(df[invalid_names].index, inplace=True)

print(df)

     University Rank                     Name of University        Location  \
0                  1                   University of Oxford  United Kingdom   
1                  2                     Harvard University   United States   
2                  3                University of Cambridge  United Kingdom   
3                  3                    Stanford University   United States   
4                  5  Massachusetts Institute of Technology   United States   
...              ...                                    ...             ...   
2336               -     University of the West of Scotland             NaN   
2337               -                  University of Windsor             NaN   
2338               -            University of Wolverhampton             NaN   
2339               -                University of Wuppertal             NaN   
2340               -    Xi’an Jiaotong-Liverpool University             NaN   

     No of student  No of student per staff Interna

<div style="text-align: justify;">
Para los campos vacíos en la columna Location, se decidió sustituir dichos valor por un valor "Unknown", a pesar de que se puede completar este campo de forma manual utilizando el nombre de la universidad.
</div>

In [24]:
df['Location'].fillna('Unknown', inplace=True)
print(df['Location'])

0       United Kingdom
1        United States
2       United Kingdom
3        United States
4        United States
             ...      
2336           Unknown
2337           Unknown
2338           Unknown
2339           Unknown
2340           Unknown
Name: Location, Length: 2233, dtype: object


<div style="text-align: justify;">
Para la variable "No of student" se notaron ciertas inconsistencias en los valores, ya que existen universidades con un valor no entero de estudiantes, además de ello se encontraron observaciones con valores muy bajos. En este caso el primer paso fue modificar el formato de representación a valores enteros, posteriormente se puede calcular un promedio de estudiantes de las universidades de un mismo país, de esa forma se pueden corregir las inconsistencias mencionadas anteriormente.
</div>

In [43]:
df['No of student'] = df['No of student'].str.replace(',','')
df['No of student'] = pd.to_numeric(df['No of student'], errors = 'coerce')
print(df)

AttributeError: Can only use .str accessor with string values!

In [44]:
def lessThan1000(value):
    return value < 1000

valid_df = df[~df['No of student'].apply(lessThan1000)]

avg_students_by_location = valid_df.groupby('Location')['No of student'].mean()

def replaceWithAVG(row):
    if lessThan1000(row['No of student']):
        if row['Location'] == 'Unknown':
            return row['No of student'] #Mantener valores originales para paises desconocidos
        else:
            return avg_students_by_location.get(row['Location'], row['No of student'])
    else:
        return row['No of student']
    
df['No of student'] = df.apply(replaceWithAVG, axis = 1)

print(df['No of student'])

0       20965.0
1       21887.0
2       20185.0
3       16164.0
4       11415.0
         ...   
2336        NaN
2337        NaN
2338        NaN
2339        NaN
2340        NaN
Name: No of student, Length: 2233, dtype: float64


<div style="text-align: justify;">
Se pueden dividir en 2 columnas el ratio entre estudiantes de sexo masculino y femenino, de forma que sea más cómodo de procesar.
</div>

In [49]:
df['Female:Male Ratio'] = df['Female:Male Ratio'].str.replace(" : ", "")

def getFemaleRatio(ratio):
    if pd.isna(ratio):
        return None
    else:
        return str(ratio)[:2]
    
def getMaleRatio(ratio):
    if pd.isna(ratio):
        return None
    else:
        return str(ratio)[-2:]

df['Female students %'] = df['Female:Male Ratio'].apply(getFemaleRatio)
df['Male students %'] = df['Female:Male Ratio'].apply(getMaleRatio)

df = df.drop('Female:Male Ratio', axis=1)
print(df)

     University Rank                     Name of University        Location  \
0                  1                   University of Oxford  United Kingdom   
1                  2                     Harvard University   United States   
2                  3                University of Cambridge  United Kingdom   
3                  3                    Stanford University   United States   
4                  5  Massachusetts Institute of Technology   United States   
...              ...                                    ...             ...   
2336               -     University of the West of Scotland         Unknown   
2337               -                  University of Windsor         Unknown   
2338               -            University of Wolverhampton         Unknown   
2339               -                University of Wuppertal         Unknown   
2340               -    Xi’an Jiaotong-Liverpool University         Unknown   

      No of student  No of student per staff Intern